# Graph visualisation - social graph of Wikipedia editors

This notebook presents the full pipeline used to construct and analyse a user–user interaction graph of Wikipedia editors.

* **Nodes** represent individual users (editors).
* **Edges** connect pairs of users who have edited at least one common article.
* **Edge weights** correspond to the number of distinct articles co-edited, capturing the intensity of shared editing activity
(a normalized alternative based on Jaccard similarity can be introduced at a later stage).
* **Node attributes** include editor metadata such as user type, total number of edits, and average weaponising ratio.

Community detection is performed using the **Leiden algorithm**, and the resulting clusters are visualized with the **Distributed Recursive Layout (DRL)**, which is well suited for large graphs.

The detected clusters correspond to communities of editors with overlapping article portfolios, highlighting patterns of shared editorial focus and potential coordination.

In [1]:
import pandas as pd
from itertools import combinations
from collections import Counter
import igraph as ig
import leidenalg
import numpy as np

In [2]:
# import the dataset. We will first test on the small database of 2336 unique users.
df = pd.read_csv('../datas/final/small_db_preprocess.csv')
df

,article,user,date,comment,llm_output,weaponised,year,user_type
0,COVID-19 pandemic in Ukraine,Agathoclea,2020-03-11T20:56:06Z,removed [[Category:2019–20 coronavirus outbrea...,"Changed the category from ""2019–20 coronavirus...",Not Weaponised,2020,Registered
1,History of Ukraine,Icey,2006-05-21T14:09:22Z,/* Further reading */ Disambiguation link repa...,Changed the reference format for Andrew Wilson...,Not Weaponised,2006,Registered
2,History of Ukraine,Irpen,2006-06-06T21:00:08Z,"this whole section doesn't belong here, speara...","Removed a section titled ""Ukraine and Nuclear ...",Not Weaponised,2006,Registered
3,History of Ukraine,193.60.161.100,2006-05-23T11:39:26Z,NaN,"Changed ""beyond"" to ""gayniss"" in the context o...",Not Weaponised,2006,Anonymous (IP)
4,History of Ukraine,Irpen,2006-06-14T17:49:44Z,revert to myself,Removed a POV (point of view) section regardin...,Not Weaponised,2006,Registered
...,...,...,...,...,...,...,...,...
6917,COVID-19 pandemic in Ukraine,LSGH,2021-05-06T05:57:13Z,Updating number of cases in infobox,"Changed confirmed cases, recovery cases, death...",Not Weaponised,2021,Registered
6918,2014 pro-Russian unrest in Ukraine,Garik 11,2014-04-08T17:05:13Z,/* Latvian citizen arrested */ more detail abo...,Changed the description of a Latvian citizen b...,Not Weaponised,2014,Registered
6919,2014 pro-Russian unrest in Ukraine,Aleksandr Grigoryev,2014-04-19T15:24:13Z,/* Kidnapping of Ukrainian officials */ update,The change made in this revision is the additi...,Weaponised,2014,Registered
6920,Censuses in Ukraine,Aleksandr Grigoryev,2012-11-17T17:14:35Z,/* External links */ update,"Added a template for ""Ukraine topics"" and a ca...",Not Weaponised,2012,Registered


In [3]:
df["is_weaponised"] = (df["weaponised"].str.lower().str.strip() == "weaponised").astype(int)

# we group by paris of user editing an article and Computes fraction of weaponised edits per user–article pair
# 0.0 is considered as neutral behaviour
# 1.0 is considered as fully weaponised on that article
# intermediate is mixed behaviour
weaponising_ratio_df = (
    df.groupby(["user", "article", "user_type"])
      .agg(
          total_edits=("is_weaponised", "count"),
          weaponised_edits=("is_weaponised", "sum")
      )
      .reset_index()
)

weaponising_ratio_df["weaponising_ratio"] = (
    weaponising_ratio_df["weaponised_edits"] / weaponising_ratio_df["total_edits"]
)

# transform the article name in a number using cat.codes
weaponising_ratio_df["article_id"] = (
    weaponising_ratio_df["article"].astype("category").cat.codes
)

df_graph = weaponising_ratio_df.rename(
    columns={"total_edits": "n_edits"}
)

# we get rid of the Bot users. 
df_graph = df_graph[df_graph["user_type"] != "Bot"]


In [4]:
pairs = [] # list that will contains set of two users that co-edited the same article

# for each article, we get the set of users that edited it. Then for every user in that set, we create unordered combinaison of 2 users.
for article, group in df_graph.groupby("article_id"):
    users = group["user"].unique()
    for u1, u2 in combinations(sorted(users), 2):
        pairs.append((u1, u2))

coedit_counts = Counter(pairs)

edges_df = pd.DataFrame(
    [(u1, u2, w) for (u1, u2), w in coedit_counts.items()],
    columns=["user1", "user2", "coedit_count"]
)
# Counts how many different articles each pair co-edited (since pairs were added once per article). This count will become the edge weight.

# We can remove weak connections (coedit_count < 2)
# A connection >= 3 get rid of all the IP user.
edges_df = edges_df[edges_df["coedit_count"] >= 1].reset_index(drop=True)

user_stats = (
    df_graph.groupby("user")
    .agg(
        total_edits=("n_edits", "sum"),
        mean_weaponising_ratio=("weaponising_ratio", "mean")
    )
    .reset_index()
)

# Graph from tuple list (handles string names properly)
g = ig.Graph.TupleList(
    edges_df.itertuples(index=False, name=None),  # ensures tuple of (user1, user2, weight)
    weights=True,
    directed=False
)

# Add node attributes
user_dict = user_stats.set_index("user").to_dict("index")

g.vs["total_edits"] = [user_dict.get(v["name"], {}).get("total_edits", 0) for v in g.vs]
g.vs["weaponising_ratio"] = [user_dict.get(v["name"], {}).get("mean_weaponising_ratio", 0) for v in g.vs]

partition = leidenalg.find_partition(
    g, leidenalg.ModularityVertexPartition, weights=g.es["weight"]
)
g.vs["cluster"] = partition.membership

print(f"Graph built: {g.vcount()} nodes, {g.ecount()} edges, {len(set(partition.membership))} clusters.")

Graph built: 2243 nodes, 287323 edges, 17 clusters.


In [13]:
# Save to GraphML format (recommended for Gephi)
wanna_save = True
if wanna_save:
    g.write_graphml("../plots/user_network_min1.graphml")
    print("Graph exported")

Graph exported


## Cluster summaries & quantitative analysis

| Question                               | How                                      |
| -------------------------------------- | ---------------------------------------- |
| How big is each cluster?               | node counts                              |
| How dense is it?                       | internal edge density                    |
| Are clusters isolated?                 | inter-cluster edge weight                |
| Is weaponisation unevenly distributed? | mean / distribution of weaponising ratio |
| Do clusters differ by user type?       | registered vs IP                         |


In [14]:
nodes_df = pd.DataFrame({
    "user": g.vs["name"],
    "cluster": g.vs["cluster"],
    "total_edits": g.vs["total_edits"],
    "weaponising_ratio": g.vs["weaponising_ratio"]
})

nodes_df

,user,cluster,total_edits,weaponising_ratio
0,*Kat*,4,1,0.0
1,124.19.38.60,4,1,0.0
2,124.190.40.8,4,10,0.1
3,128.189.147.85,4,1,0.0
4,128.214.107.187,4,5,0.0
...,...,...,...,...
2238,Tibet2014,3,1,0.0
2239,Tyrol5,3,2,0.0
2240,Underlying lk,3,1,0.0
2241,XXN,3,1,0.0


In [7]:
# cluster size + activity volume

cluster_size_df = (
    nodes_df
    .groupby("cluster")
    .agg(
        n_users=("user", "count"),
        total_edits=("total_edits", "sum"),
        mean_edits_per_user=("total_edits", "mean")
    )
    .reset_index()
    .sort_values("n_users", ascending=False)
)

cluster_size_df

,cluster,n_users,total_edits,mean_edits_per_user
0,0,470,1203,2.559574
1,1,416,1537,3.694712
2,2,333,773,2.321321
3,3,173,550,3.179191
4,4,158,501,3.170886
5,5,153,291,1.901961
6,6,148,1177,7.952703
7,7,123,150,1.219512
8,8,97,189,1.948454
9,9,68,124,1.823529


In [8]:
# Weaponisation profile per cluster

cluster_weapon_df = (
    nodes_df
    .groupby("cluster")
    .agg(
        mean_weaponising_ratio=("weaponising_ratio", "mean"),
        median_weaponising_ratio=("weaponising_ratio", "median"),
        
        # number of users with strong weaponisation behaviour
        high_weaponisers=("weaponising_ratio", lambda x: (x >= 0.5).sum())
    )
    .reset_index()
)

cluster_weapon_df

,cluster,mean_weaponising_ratio,median_weaponising_ratio,high_weaponisers
0,0,0.212741,0.0,105
1,1,0.331374,0.0,146
2,2,0.169705,0.0,59
3,3,0.256821,0.0,44
4,4,0.114420,0.0,16
5,5,0.103274,0.0,18
6,6,0.065310,0.0,8
7,7,0.114983,0.0,15
8,8,0.088739,0.0,9
9,9,0.084395,0.0,6


In [9]:
# each row describes the distribution of weaponising behaviour among users in a cluster.
cluster_weapon_dist = (
    nodes_df
    .groupby("cluster")["weaponising_ratio"]
    .describe()
    .reset_index()
)

cluster_weapon_dist


,cluster,count,mean,std,min,25%,50%,75%,max
0,0,470.0,0.212741,0.355826,0.0,0.0,0.0,0.333333,1.0
1,1,416.0,0.331374,0.427925,0.0,0.0,0.0,1.000000,1.0
2,2,333.0,0.169705,0.349963,0.0,0.0,0.0,0.000000,1.0
3,3,173.0,0.256821,0.393759,0.0,0.0,0.0,0.500000,1.0
4,4,158.0,0.114420,0.276250,0.0,0.0,0.0,0.000000,1.0
5,5,153.0,0.103274,0.276789,0.0,0.0,0.0,0.000000,1.0
6,6,148.0,0.065310,0.211238,0.0,0.0,0.0,0.000000,1.0
7,7,123.0,0.114983,0.312240,0.0,0.0,0.0,0.000000,1.0
8,8,97.0,0.088739,0.279467,0.0,0.0,0.0,0.000000,1.0
9,9,68.0,0.084395,0.231319,0.0,0.0,0.0,0.000000,1.0


In [10]:
# High internal / low external : cohesive, well-separated community
# High external : bridge-like or diffuse cluster

cluster_ids = sorted(set(g.vs["cluster"]))

rows = []

for c in cluster_ids:
    nodes_in_c = [v.index for v in g.vs if v["cluster"] == c]
    nodes_not_c = [v.index for v in g.vs if v["cluster"] != c]

    internal_edges = g.es.select(_within=nodes_in_c)
    external_edges = g.es.select(_between=(nodes_in_c, nodes_not_c))

    rows.append({
        "cluster": c,
        "internal_edge_weight": sum(internal_edges["weight"]),
        "external_edge_weight": sum(external_edges["weight"]),
        "n_nodes": len(nodes_in_c)
    })

cluster_structure_df = pd.DataFrame(rows)
cluster_structure_df

,cluster,internal_edge_weight,external_edge_weight,n_nodes
0,0,110513,20772,470
1,1,86667,16733,416
2,2,10835,6909,333
3,3,9095,8169,173
4,4,12416,7005,158
5,5,4976,3625,153
6,6,9513,2316,148
7,7,3727,1591,123
8,8,4656,982,97
9,9,936,463,68


## Deep Analysis of clusters 

From the quantitative metrics, we cannot identitfy some echo chambers. We have more a kind of core-periphery relationship. We want to retrieve the articles and the specific user names to detect :
* thematic specilization
* temporal editing
* editorial roles
* ...

In [11]:
user_cluster_df = pd.DataFrame({
    "user": g.vs["name"],
    "cluster": g.vs["cluster"]
})

df_clustered = df_graph.merge(
    user_cluster_df,
    on="user",
    how="inner"
)

core_articles = (
    df_clustered[df_clustered["cluster"] == 0]
    .groupby("article")
    .agg(
        n_users=("user", "nunique"),
        total_edits=("n_edits", "sum")
    )
    .reset_index()
    .sort_values(["n_users", "total_edits"], ascending=False)
)

core_articles.head(15)

,article,n_users,total_edits
28,History of Ukraine,470,877
0,2004 Ukrainian presidential election,14,62
25,History of Christianity in Ukraine,11,52
21,Football in Ukraine,9,11
20,Flag of Ukraine,7,11
1,2014 pro-Russian unrest in Ukraine,6,59
5,Bessarabia,6,11
3,Alexander II of Russia,5,6
9,Christianity in Russia,5,6
6,Bukovina,4,30


In [12]:
peripheral_cluster = 4

peripheral_articles = (
    df_clustered[df_clustered["cluster"] == peripheral_cluster]
    .groupby("article")
    .agg(
        n_users=("user", "nunique"),
        total_edits=("n_edits", "sum")
    )
    .reset_index()
    .sort_values(["n_users", "total_edits"], ascending=False)
)

core_article_set = set(core_articles["article"])
peripheral_articles["also_in_core"] = (
    peripheral_articles["article"].isin(core_article_set)
)

peripheral_articles

,article,n_users,total_edits,also_in_core
0,2004 Ukrainian presidential election,158,328,True
17,History of Ukraine,3,57,True
9,Crimea,3,15,False
8,Communist Party of the Soviet Union,3,3,True
6,COVID-19 pandemic in Ukraine,2,37,True
12,Flag of Ukraine,2,13,True
7,Christianity in Russia,2,3,True
4,Bessarabia,2,2,True
1,2014 pro-Russian unrest in Ukraine,1,28,True
10,Economy of Ukraine,1,3,True
